# HW2 Association Rules

`NCU MIS ECT` `106403551` `呂晟維`
1. 使⽤weka回答以下問題：
    - (a)請嘗試著修改 adult.csv 的欄位與上圖相同，並轉換成 arff 檔使 weka可以執⾏ Association Rule，請說明使⽤⽅法以及解釋原來的檔案不能執⾏的原因︖(10%)
    - (b) 請將 numRules 設成 5 和 10，其各別執⾏後的 Minimum support 為何，請比較兩者並說明造成其差異的原因。(15%)
    - (c) 將 numRule 設成 10，列出前 5 條 rule(15%)
    - (d) 如何在 Associator output 產⽣ Itemset，請截圖說明並附上 Itemset 結果。(15%)


2. 使⽤python回答以下問題：
    - (e) 使⽤已修改過的 adult.csv 檔，使⽤ Apriori 演算法進⾏分析,設定 confidence = 0.9、minimum support = 0.2，過程中對所有重要程式步驟進⾏截圖並加以說明，越詳盡越好。(15%)
    - (f) 產⽣與 (c) ⼩題⼀樣的結果，列出前五條best rules，截圖並加以說明(15%)

## 1. Weka

參考資料：[以Weka對資料集進行關聯式規則之實作](https://medium.com/@bt2011aa/%E4%BB%A5weka%E5%B0%8D%E8%B3%87%E6%96%99%E9%9B%86%E9%80%B2%E8%A1%8C%E9%97%9C%E8%81%AF%E5%BC%8F%E8%A6%8F%E5%89%87%E4%B9%8B%E5%AF%A6%E4%BD%9C-e7a87c2005a9)、[Mining Association Rule with WEKA Explorer](https://storm.cis.fordham.edu/~yli/documents/CISC4631Spring16/Weka_LabTwo.pdf)

### (a) 請嘗試著修改 adult.csv 的欄位與上圖相同，並轉換成 arff 檔使 weka可以執行 Association Rule，請說明使用方法以及解釋原來的檔案不能執行的原因? (10%)

#### 純Python作法
我也手刻一隻python程式`Csv to Arff.ipynb`可以將.csv轉換成.arff，因為考量通用性，所有的attribute都會是nominal型態。沒有提供改名功能。

#### 純Weka作法
一開始我們只有adult.csv檔案，匯入時發現Association Rules的start鍵按不下去，因為csv裡面gender的欄位資料只有0和1；0代表Male，1代表Female，但是weka判斷此欄位為numeric型態，所以執行不了。
![](https://i.imgur.com/1FHa929.png)
![](https://i.imgur.com/lymVlgK.png)

因此需要先將numeric型態，轉成nominal型態。Weka有內鍵功能來轉換attribute的型態，我們先選取gender欄位，點選filter > nonsupervise > numeric to nominal按下執行即可。如此一來就可將0和1轉成nominal的形式。
```python
@attribute gender {0, 1}
```
![](https://i.imgur.com/tRCPfLX.png)

接下來，由於gender的值還是0和1，雖然可以run但可讀性還是差了一些，所以接下來我們再把0和1改名成Male和Female，選取filter > nonsupervise > rename nominal values。按照下圖輸入參數。最後再save成.arff檔即可。
```python
@attribute gender {Male, Female}
```
![](https://i.imgur.com/nmTavlE.png)
轉檔後的結果
![](https://i.imgur.com/3ZcpRxR.png)

### (b) 請將 numRules 設成 5 和 10，其各別執行後的 Minimum support 為何，請比較兩者並說明造成其差異的原因。(15%)

#### numRules設成5的結果：Minimum support為0.25
```js
Minimum support: 0.25 (11508 instances)
Minimum metric <confidence>: 0.9
Number of cycles performed: 15
```
#### numRules設成10的結果：Minimum support為0.2
```js
Minimum support: 0.2 (9207 instances)
Minimum metric <confidence>: 0.9
Number of cycles performed: 16
```
解讀一下這份報表，weka一開始從support = 1找confidence > 0.9的rules。演算法一共重複執行了Number of cycles performed次才停下來，停下來的情形有2種，一是找到指定數目筆的rules(5或10個)，二是Minimum support下降至指定值(0.2)時還未找齊rules。

當numrule = 5時，演算法重複執行了15次便集齊5個rules了(大於等於5個)，第15次的Minimum support是0.25；而要再進入第16次cycle才能找到10個rules，Minimum support為0.25 - 0.05 = 0.2。

另外由於rules的排序是照confidence的大小來排，因此每條規則的rank會不太一樣，像是rules5的第一條規則在rule10中是第二條。

* **Rule 5的全部截圖**
![](https://i.imgur.com/2Cn2XQt.png)
* **Rule 10的全部截圖**
![](https://i.imgur.com/hKSdVgh.png)

### (c) 將 numRule 設成 10，列出前 5 條 rule(15%)

- 這5個rules的lift值(提升度)都 > 1，Conviction(conv確信度)值也很大，代表前因和後果之間的關係相當密切。
- 這5個rules的support count也都在10000筆資料上下，在46000筆樣本中算是多數了，因此結果值得信賴。
    
#### Rules如下：
    
1. marital-status=Never-married hours-per-week=20-40 9669 ==> income= <=50K 9368    
<conf:(0.97)> lift:(1.29) lev:(0.05) [2098] conv:(7.94)
```
→ 沒有結婚 ∩ 工時20~40小時的人，有0.97的比例，他們的收入小於50K
```           
2. workclass=Private marital-status=Never-married 12243 ==> income= <=50K 11755    
<conf:(0.96)> lift:(1.28) lev:(0.06) [2549] conv:(6.21)
```
→ 在私人機構工作 ∩ 沒有結婚的人，有0.96的比例，他們的收入小於50K
```             
3. workclass=Private marital-status=Never-married race=White 10134 ==> income= <=50K 9702    
<conf:(0.96)> lift:(1.27) lev:(0.05) [2082] conv:(5.81)
```
→ 在私人機構工作 ∩ 沒有結婚 ∩ 是白人，有0.96的比例，他們的收入小於50K
```      
4. marital-status=Never-married 14875 ==> income= <=50K 14153    
<conf:(0.95)> lift:(1.27) lev:(0.06) [2968] conv:(5.1)
```
→ 沒有結婚的人，有0.95的比例，他們的收入小於50K
```  
5. marital-status=Never-married race=White 12228 ==> income= <=50K 11590    
<conf:(0.95)> lift:(1.26) lev:(0.05) [2396] conv:(4.75)
```
→ 沒有結婚 ∩ 是白人，有0.95的比例，他們的收入小於50K
```  


### (d) 如何在 Associator output 產生 Itemset，請截圖說明並附上 Itemset 結果。(15%)

將output itemset的選項設成True即可看到frequent itemset的output。Itemset會依support值高到低(從1.0開始)尋找support值 >= min_sup 的集合，每個循環會從一個元素且符合min_sup的集合找起，再找兩個元素的集合、三個元素的集合、四個元素的集合...。再依每種集合(itemsets)的confidence來找尋規則，confidence即是條件機率的意思，當confidence >= 0.9便是一條rule。

![](https://i.imgur.com/K2Q42LF.png)

Rule10的輸出如下：
```js
Generated sets of large itemsets:

Size of set of large itemsets L(1): 15

Large Itemsets L(1):
age=20-30 11487
age=30-40 12538
age=40-50 10182
workclass=Private 33906
education=HS-grad 14972
education=Some-college 10036
marital-status=Never-married 14875
marital-status=Married-civ-spouse 21451
race=White 39444
gender=Male 31114
gender=Female 14919
hours-per-week=20-40 28350
hours-per-week=40-60 12403
income=<=50K 34611
income=>50K 11422

Size of set of large itemsets L(2): 38

Large Itemsets L(2):
age=20-30 workclass=Private 9649
age=20-30 race=White 9650
age=20-30 income=<=50K 10513
age=30-40 workclass=Private 9370
age=30-40 race=White 10636
workclass=Private education=HS-grad 11682
...

Size of set of large itemsets L(3): 29

Large Itemsets L(3):
workclass=Private education=HS-grad race=White 9907
workclass=Private education=HS-grad income=<=50K 9983
...

Size of set of large itemsets L(4): 8

Large Itemsets L(4):
workclass=Private marital-status=Never-married race=White income=<=50K 9702
workclass=Private marital-status=Married-civ-spouse race=White gender=Male 11625
...
```

## 2. 使用Python回答以下問題：
    

### (e) 使用已修改過的 adult.csv 檔，使用 Apriori 演算法進行分析,設定 confidence = 0.9、minimum support = 0.2，過程中對所有重要程式步驟進行截圖並加以說明，越詳盡越好。(15%)

可以直接跳至程式的.ipynb，有列出所有的參考資料和詳盡程式解說。

我們程式分成2步驟，資料前處理和演算法執行，最後我們會解析結果：

- **整理資料**

利用pandas讀csv檔，apriori的輸入和輸出都是list型態的資料結構，所以把df轉成list。
![](https://i.imgur.com/LuM3Fyi.png)
![](https://i.imgur.com/jxbQQ4i.png)


- **apriori演算法**

重點程式碼是apriori方法，輸入值有min_support= 0.2, min_confidence= 0.9。
```python
from apyori import apriori

#建立rule, 設定參數 #變成list
rules = list(apriori(data, min_support= 0.2, min_confidence= 0.9))
```
![](https://i.imgur.com/2qPKUEt.png)

- **解析結果**

根據以下屬性可以整理出得出的規則們
    - items_base – 關聯規則中的分母項集 (ex買了啤酒)
    - items_add –  關聯規則中的分子項集 (ex跟著買尿布)
![](https://i.imgur.com/SeJzhmD.png)

### (f) 產生與 (c) 小題一樣的結果，列出前五條best rules，截圖並加以說明(15%)

我們發現python程式的結果和weka的結果一模一樣，第一條rule的confidence = 0.968870 = 9368 / 9669，因為weka只四捨五入到0.97，若把樣本數相除則和python結果相符。關聯的規則是"沒有結婚 ∩ 工時20~40小時的人，有0.97的比例，他們的收入小於50K"。

其他四項不加贅述了。偷偷一提python的apriori產生的rules的結果超級難解讀(´_ゝ`)，規則也都沒有直接列出的因果關係。下表是額外查資料才做出的關係。

![](https://i.imgur.com/nAhLMpx.png)